In [157]:
import pandas as pd
import numpy as np
import itertools
import random
import json

In [158]:
df = pd.read_csv('twitch_gamers/large_twitch_edges.csv')

In [159]:
edges = df['numeric_id_1'].value_counts()
lst_100 = edges[:100].index

/var/folders/s8/571wktkn49b_yr1hmss84z5c0000gn/T/ipykernel_47354/2256113037.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  lst_100 = edges[:100].index


In [160]:
pairs = list(itertools.combinations(lst_100, 2))

In [161]:
df_top_pairs = pd.DataFrame(pairs, columns=['streamer1', 'streamer2'])
df_top_pairs
#top 100 streamers paired with each other.

,streamer1,streamer2
0,61862,125642
1,61862,32338
2,61862,71050
3,61862,110345
4,61862,64605
...,...,...
4945,114786,44335
4946,114786,160608
4947,144089,44335
4948,144089,160608


In [162]:
grouped = df.groupby(['numeric_id_1'])
grouped = grouped.agg(set).reset_index()
#see each nodes' connections

In [163]:
grouped = grouped[grouped['numeric_id_1'].isin(lst_100)]
grouped = grouped.reset_index(drop=True)
#only the top 100 streamers connections

In [164]:
neighbors = grouped.set_index('numeric_id_1').to_dict()['numeric_id_2']

In [165]:
pairs_shared = {}
for pair in pairs:
    pairs_shared[pair] = len(neighbors[pair[0]].intersection(neighbors[pair[1]]))
    # print(pair,len(neighbors[pair[0]].intersection(neighbors[pair[1]])))

In [166]:
streamer_df = pd.DataFrame(pairs_shared.keys(), columns=['streamer_1','streamer_2'])
streamer_df['num_shared_viewers'] = pairs_shared.values()

In [167]:
streamer_df.sort_values

<bound method DataFrame.sort_values of       streamer_1  streamer_2  num_shared_viewers
0          61862      125642               17328
1          61862       32338                6879
2          61862       71050               16021
3          61862      110345               10760
4          61862       64605                9988
...          ...         ...                 ...
4945      114786       44335                 399
4946      114786      160608                 407
4947      144089       44335                 379
4948      144089      160608                 299
4949       44335      160608                 296

[4950 rows x 3 columns]>

In [168]:
lst_100 = list(lst_100)

In [169]:
nodes =[]
jsons = {}
links = []
rank = 1
for streamer_id in lst_100:
    tmp = {}
    tmp['id']= streamer_id
    tmp['name'] = 'Rank #' + str(rank)
    nodes.append(tmp)
    rank += 1

for idx, row in streamer_df.iterrows():
    tmp = {}
    # added to reduce number of links
    if row['streamer_1'] == row['streamer_2'] or row['num_shared_viewers'] < 1000:
        continue
    tmp['source'] = int(row['streamer_1'])
    tmp['target'] = int(row['streamer_2'])
    tmp['value'] = int(row['num_shared_viewers'])
    links.append(tmp)

jsons['nodes'] = nodes
jsons['links'] = links
json_data = json.dumps(jsons)
json_data

'{"nodes": [{"id": 61862, "name": "Rank #1"}, {"id": 125642, "name": "Rank #2"}, {"id": 32338, "name": "Rank #3"}, {"id": 71050, "name": "Rank #4"}, {"id": 110345, "name": "Rank #5"}, {"id": 64605, "name": "Rank #6"}, {"id": 52703, "name": "Rank #7"}, {"id": 155127, "name": "Rank #8"}, {"id": 60588, "name": "Rank #9"}, {"id": 123018, "name": "Rank #10"}, {"id": 6250, "name": "Rank #11"}, {"id": 128864, "name": "Rank #12"}, {"id": 78367, "name": "Rank #13"}, {"id": 32283, "name": "Rank #14"}, {"id": 152296, "name": "Rank #15"}, {"id": 5507, "name": "Rank #16"}, {"id": 31982, "name": "Rank #17"}, {"id": 144643, "name": "Rank #18"}, {"id": 111372, "name": "Rank #19"}, {"id": 139587, "name": "Rank #20"}, {"id": 94979, "name": "Rank #21"}, {"id": 85877, "name": "Rank #22"}, {"id": 88248, "name": "Rank #23"}, {"id": 161362, "name": "Rank #24"}, {"id": 58773, "name": "Rank #25"}, {"id": 7329, "name": "Rank #26"}, {"id": 167147, "name": "Rank #27"}, {"id": 26309, "name": "Rank #28"}, {"id": 10

In [170]:
with open('node_data.json', 'w', encoding='utf-8') as f:
    json.dump(jsons, f, ensure_ascii=False, indent=4)

In [3]:
features = pd.read_csv('twitch_gamers/large_twitch_features.csv')

In [13]:
features.head()

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
0,7879,1,969,2016-02-16,2018-10-12,0,0,EN,1
1,500,0,2699,2011-05-19,2018-10-08,1,0,EN,0
2,382502,1,3149,2010-02-27,2018-10-12,2,0,EN,1
3,386,0,1344,2015-01-26,2018-10-01,3,0,EN,0
4,2486,0,1784,2013-11-22,2018-10-11,4,0,EN,0


In [70]:
twitch_lang = list(features['language'].unique())
twitch_lang

['EN',
 'FR',
 'KO',
 'JA',
 'RU',
 'PL',
 'DE',
 'ES',
 'IT',
 'PT',
 'OTHER',
 'TR',
 'ZH',
 'SV',
 'NL',
 'TH',
 'CS',
 'DA',
 'HU',
 'FI',
 'NO']

In [75]:
import pycountry
from babel import languages

langSym = {}
for country in pycountry.countries:
    langSym[country.name] = country.alpha_2


In [79]:
langSym

{'Aruba': 'AW',
 'Afghanistan': 'AF',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Åland Islands': 'AX',
 'Albania': 'AL',
 'Andorra': 'AD',
 'United Arab Emirates': 'AE',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'American Samoa': 'AS',
 'Antarctica': 'AQ',
 'French Southern Territories': 'TF',
 'Antigua and Barbuda': 'AG',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Burundi': 'BI',
 'Belgium': 'BE',
 'Benin': 'BJ',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Burkina Faso': 'BF',
 'Bangladesh': 'BD',
 'Bulgaria': 'BG',
 'Bahrain': 'BH',
 'Bahamas': 'BS',
 'Bosnia and Herzegovina': 'BA',
 'Saint Barthélemy': 'BL',
 'Belarus': 'BY',
 'Belize': 'BZ',
 'Bermuda': 'BM',
 'Bolivia, Plurinational State of': 'BO',
 'Brazil': 'BR',
 'Barbados': 'BB',
 'Brunei Darussalam': 'BN',
 'Bhutan': 'BT',
 'Bouvet Island': 'BV',
 'Botswana': 'BW',
 'Central African Republic': 'CF',
 'Canada': 'CA',
 'Cocos (Keeling) Islands': 'CC',
 'Switzerland': 'CH',
 'Chile': 'CL',
 'China': 'CN',
 "Côte d'Iv

In [83]:
country_lang = {}
for country in pycountry.countries:
    # langSym[country.name] = country.alpha_2
    most_pop_lang = list(languages.get_territory_language_info(country.alpha_2).keys())[0].upper()
    if most_pop_lang in twitch_lang:
        if country.name not in 
        country_lang[country.name] = most_pop_lang
    else:
        country_lang[country.name] = 'OTHER'

In [106]:
#Should redo this so that it doesn't replace the name but just adds the language
renamed = []
with open('world.geo.json', 'r') as file:
    data = json.load(file)

# Update the country names in the JSON data
for feature in data['features']:
    country_name = feature['properties']['name']
    if country_name in country_lang:
        feature['properties']['name'] = country_lang[country_name]
    else:
        renamed.append(country_name)

# Write the updated JSON data to a new file
with open('updated_worldgeo.json', 'w', encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=1)

In [107]:
renamed #had to change these manually in renamed_worldgeo.json cause name doesn't match

['French Southern and Antarctic Lands',
 'The Bahamas',
 'Bolivia',
 'Brunei',
 'Ivory Coast',
 'Democratic Republic of the Congo',
 'Republic of the Congo',
 'Northern Cyprus',
 'Czech Republic',
 'Falkland Islands',
 'England',
 'Guinea Bissau',
 'Iran',
 'South Korea',
 'Kosovo',
 'Laos',
 'Moldova',
 'Macedonia',
 'North Korea',
 'Russia',
 'Somaliland',
 'Republic of Serbia',
 'Swaziland',
 'Syria',
 'East Timor',
 'Taiwan',
 'United Republic of Tanzania',
 'USA',
 'Venezuela',
 'Vietnam',
 'West Bank']

In [138]:
#example changed this manually in renamed_worldgeo.json
country_lang['North Macedonia']

'OTHER'

In [144]:
game_df = pd.read_csv("game_data/Twitch_game_data.csv", encoding = 'latin1')

In [153]:
game_df[game_df['Year'] == 2017]

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
2400,1,League of Legends,1,2017,99332369,1363324,379399,2868,129423,133690,1834,72.86
2401,2,Counter-Strike: Global Offensive,1,2017,72376812,694606,1233293,2177,110118,97411,934,104.20
2402,3,Dota 2,1,2017,39036428,359580,231960,877,33380,52538,483,108.56
2403,4,Hearthstone,1,2017,37651061,179394,97854,415,24800,50674,241,209.88
2404,5,Overwatch,1,2017,25700463,972764,88338,2423,149853,34590,1309,26.42
...,...,...,...,...,...,...,...,...,...,...,...,...
4795,196,Microsoft Flight Simulator X,12,2017,165600,13898,2997,52,1049,222,18,11.92
4796,197,Steep,12,2017,163904,4717,7951,28,1773,220,6,34.75
4797,198,Yakuza Kiwami,12,2017,163640,3464,5436,14,518,220,4,47.24
4798,199,Dreadnought,12,2017,163610,4758,7277,23,615,220,6,34.39
